In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
from tqdm import tqdm
import os.path as op

from mod import load_data, alt_theme

from warnings import simplefilter
simplefilter('ignore')

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
bot_names = pd.read_csv('./bot_names.csv')

In [4]:
fmt_date = "{:%Y-%m-%d}"

start_date = None
end_date = None

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [5]:
# Parameters
renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples"]


In [6]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [7]:
comments, issues, prs = load_data('../data/', github_orgs=github_orgs)
comments = comments.query('author not in @bot_names').drop_duplicates()
issues = issues.query('author not in @bot_names').drop_duplicates()
prs = prs.query('author not in @bot_names').drop_duplicates()

In [8]:
if start_date is None:
    start_date = comments['updatedAt'].min()
    
if end_date is None:
    end_date = comments['updatedAt'].max()

In [9]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

In [10]:
def highlighted_line_plot(data, x, y, color, title, domain=["2019-05-01", "2019-08-01"], legend=True):
    ch = alt.Chart(data=data, width=1000, title=title)

    highlight = alt.selection(type='single', on='mouseover',
                              fields=[color], nearest=True)

    ln = ch.mark_line(clip=True).encode(
        x=alt.X(x, scale=alt.Scale(domain=domain)),
        y=y,
        color=alt.Color(color, legend=legend),
        opacity=alt.condition(~highlight, alt.value(.3), alt.value(1)),
        size=alt.condition(~highlight, alt.value(2), alt.value(5))
    )

    pt = ch.mark_point(clip=True).encode(
        x=alt.X(x, scale=alt.Scale(domain=domain)),
        y=y,
        color=alt.Color(color, legend=legend),
        tooltip=color,
        size=alt.value(300),
        opacity=alt.value(0)
    ).add_selection(highlight).interactive()
    return ln + pt

In [11]:
date_fields = ['updatedAt', 'createdAt', 'closedAt']
for field in date_fields:
    if field in comments.columns:
        comments[field] = pd.to_datetime(comments[field])
    if field in prs.columns:
            prs[field] = pd.to_datetime(prs[field])
    if field in issues.columns:
            issues[field] = pd.to_datetime(issues[field])

## Comments over time by people

In [12]:
time_bin = '2W'
n_authors = 20
rolling_mean_amt = 4

In [13]:
charts = []
for iorg in github_orgs:
    print(iorg)
    icomments = comments.query('org == @iorg')
    authors = icomments.groupby(['author', 'updatedAt']).agg({'id': 'count'}).reset_index()

    top_authors = authors.groupby('author').sum()['id'].sort_values(ascending=False).head(n_authors).index
    authors = authors.query('author in @top_authors')
    authors = authors.groupby('author').resample(time_bin, on='updatedAt').sum().reset_index()
    authors = authors.groupby('author').rolling(rolling_mean_amt, on='updatedAt').mean().reset_index()
    domain = [fmt_date.format(authors['updatedAt'].min()), fmt_date.format(authors['updatedAt'].max())]
    ch = highlighted_line_plot(authors, 'updatedAt', 'id', 'author', f"Comments over time for {iorg}",
                               legend=None, domain=domain)
    charts.append(ch)
alt.hconcat(*charts)

jupyterhub


jupyter-widgets


jupyter


jupyterlab


ipython


binder-examples


HConcatChart({
  hconcat: [LayerChart({
    data:          author  level_1                 updatedAt     id
    0       Carreau        0 2015-03-08 00:00:00+00:00    NaN
    1       Carreau        1 2015-03-22 00:00:00+00:00    NaN
    2       Carreau        2 2015-04-05 00:00:00+00:00    NaN
    3       Carreau        3 2015-04-19 00:00:00+00:00   2.25
    4       Carreau        4 2015-05-03 00:00:00+00:00   1.25
    ...         ...      ...                       ...    ...
    1566  yuvipanda     1566 2019-05-26 00:00:00+00:00  53.75
    1567  yuvipanda     1567 2019-06-09 00:00:00+00:00  62.75
    1568  yuvipanda     1568 2019-06-23 00:00:00+00:00  65.00
    1569  yuvipanda     1569 2019-07-07 00:00:00+00:00  59.00
    1570  yuvipanda     1570 2019-07-21 00:00:00+00:00  26.00
    
    [1571 rows x 4 columns],
    layer: [Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'author',
          legend: None,
          type: 'nominal'
        }),
        opacity: OpacityValue({
          condition: OpacityValue({
            selection: SelectionNot({
              not: 'selector001'
            }),
            value: 0.3
          }),
          value: 1
        }),
        size: SizeValue({
          condition: SizeValue({
            selection: SelectionNot({
              not: 'selector001'
            }),
            value: 2
          }),
          value: 5
        }),
        x: X({
          field: 'updatedAt',
          scale: Scale({
            domain: ['2014-08-24', '2019-09-01']
          }),
          type: 'temporal'
        }),
        y: Y({
          field: 'id',
          type: 'quantitative'
        })
      }),
      mark: MarkDef({
        clip: True,
        type: 'line'
      }),
      title: 'Comments over time for jupyterhub',
      width: 1000
    }), Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'author',
          legend: None,
          type: 'nominal'
        }),
        opacity: ValueDefWithOptionalConditionMarkPropFieldDefnumber({
          value: 0
        }),
        size: ValueDefWithOptionalConditionMarkPropFieldDefnumber({
          value: 300
        }),
        tooltip: Tooltip({
          field: 'author',
          type: 'nominal'
        }),
        x: X({
          field: 'updatedAt',
          scale: Scale({
            domain: ['2014-08-24', '2019-09-01']
          }),
          type: 'temporal'
        }),
        y: Y({
          field: 'id',
          type: 'quantitative'
        })
      }),
      mark: MarkDef({
        clip: True,
        type: 'point'
      }),
      selection: {'selector001': SelectionDef({
        fields: ['author'],
        nearest: True,
        on: 'mouseover',
        type: 'single'
      }), 'selector002': SelectionDef({
        bind: 'scales',
        encodings: ['x', 'y'],
        type: 'interval'
      })},
      title: 'Comments over time for jupyterhub',
      width: 1000
    })]
  }), LayerChart({
    data:              author  level_1                 updatedAt     id
    0     SylvainCorlay        0 2015-05-10 00:00:00+00:00    NaN
    1     SylvainCorlay        1 2015-05-24 00:00:00+00:00    NaN
    2     SylvainCorlay        2 2015-06-07 00:00:00+00:00    NaN
    3     SylvainCorlay        3 2015-06-21 00:00:00+00:00   6.50
    4     SylvainCorlay        4 2015-07-05 00:00:00+00:00  13.25
    ...             ...      ...                       ...    ...
    1467       willingc     1467 2017-03-12 00:00:00+00:00   0.00
    1468       willingc     1468 2017-03-26 00:00:00+00:00   0.00
    1469       willingc     1469 2017-04-09 00:00:00+00:00   0.00
    1470       willingc     1470 2017-04-23 00:00:00+00:00   0.25
    1471       willingc     1471 2017-05-07 00:00:00+00:00   0.50
    
    [1472 rows x 4 columns],
    layer: [Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'author',
          legend: None,
          type: 'nominal'
        }),
        

## Comments over time by repo

In [14]:
time_bin = '2W'
n_repos = 30
rolling_mean_amt = 2

In [15]:
charts = []
for iorg in github_orgs:
    print(iorg)
    icomments = comments.query('org == @iorg')
    repos = icomments.groupby(['repo', 'updatedAt']).agg({'id': 'count'}).reset_index()

    top_repos = repos.groupby('repo').sum()['id'].sort_values(ascending=False).head(n_repos).index
    repos = repos.query('repo in @top_repos')
    repos = repos.groupby('repo').resample(time_bin, on='updatedAt').sum().reset_index()
    repos = repos.groupby('repo').rolling(rolling_mean_amt, on='updatedAt').mean().reset_index()
    domain = [fmt_date.format(repos['updatedAt'].min()), fmt_date.format(repos['updatedAt'].max())]
    ch = highlighted_line_plot(repos, 'updatedAt', 'id', 'repo', f"Comments over time for {iorg}",
                               legend=None, domain=domain)
    charts.append(ch)
alt.hconcat(*charts)

jupyterhub


jupyter-widgets
jupyter


jupyterlab


ipython
binder-examples


HConcatChart({
  hconcat: [LayerChart({
    data:                         repo  level_1                 updatedAt    id
    0       alabaster-jupyterhub        0 2018-10-21 00:00:00+00:00   NaN
    1       alabaster-jupyterhub        1 2018-11-04 00:00:00+00:00   4.0
    2       alabaster-jupyterhub        2 2018-11-18 00:00:00+00:00   0.0
    3       alabaster-jupyterhub        3 2018-12-02 00:00:00+00:00   0.0
    4       alabaster-jupyterhub        4 2018-12-16 00:00:00+00:00   0.0
    ...                      ...      ...                       ...   ...
    2010  zero-to-jupyterhub-k8s     2010 2019-07-07 00:00:00+00:00  35.5
    2011  zero-to-jupyterhub-k8s     2011 2019-07-21 00:00:00+00:00  34.5
    2012  zero-to-jupyterhub-k8s     2012 2019-08-04 00:00:00+00:00  36.0
    2013  zero-to-jupyterhub-k8s     2013 2019-08-18 00:00:00+00:00  38.0
    2014  zero-to-jupyterhub-k8s     2014 2019-09-01 00:00:00+00:00  23.5
    
    [2015 rows x 4 columns],
    layer: [Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'repo',
          legend: None,
          type: 'nominal'
        }),
        opacity: OpacityValue({
          condition: OpacityValue({
            selection: SelectionNot({
              not: 'selector013'
            }),
            value: 0.3
          }),
          value: 1
        }),
        size: SizeValue({
          condition: SizeValue({
            selection: SelectionNot({
              not: 'selector013'
            }),
            value: 2
          }),
          value: 5
        }),
        x: X({
          field: 'updatedAt',
          scale: Scale({
            domain: ['2014-08-24', '2019-09-01']
          }),
          type: 'temporal'
        }),
        y: Y({
          field: 'id',
          type: 'quantitative'
        })
      }),
      mark: MarkDef({
        clip: True,
        type: 'line'
      }),
      title: 'Comments over time for jupyterhub',
      width: 1000
    }), Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'repo',
          legend: None,
          type: 'nominal'
        }),
        opacity: ValueDefWithOptionalConditionMarkPropFieldDefnumber({
          value: 0
        }),
        size: ValueDefWithOptionalConditionMarkPropFieldDefnumber({
          value: 300
        }),
        tooltip: Tooltip({
          field: 'repo',
          type: 'nominal'
        }),
        x: X({
          field: 'updatedAt',
          scale: Scale({
            domain: ['2014-08-24', '2019-09-01']
          }),
          type: 'temporal'
        }),
        y: Y({
          field: 'id',
          type: 'quantitative'
        })
      }),
      mark: MarkDef({
        clip: True,
        type: 'point'
      }),
      selection: {'selector013': SelectionDef({
        fields: ['repo'],
        nearest: True,
        on: 'mouseover',
        type: 'single'
      }), 'selector014': SelectionDef({
        bind: 'scales',
        encodings: ['x', 'y'],
        type: 'interval'
      })},
      title: 'Comments over time for jupyterhub',
      width: 1000
    })]
  }), LayerChart({
    data:                        repo  level_1                 updatedAt   id
    0                ipyleaflet        0 2014-05-25 00:00:00+00:00  NaN
    1                ipyleaflet        1 2014-06-08 00:00:00+00:00  0.5
    2                ipyleaflet        2 2014-06-22 00:00:00+00:00  0.0
    3                ipyleaflet        3 2014-07-06 00:00:00+00:00  0.0
    4                ipyleaflet        4 2014-07-20 00:00:00+00:00  0.0
    ..                      ...      ...                       ...  ...
    652  widget-ts-cookiecutter      652 2019-03-10 00:00:00+00:00  0.5
    653  widget-ts-cookiecutter      653 2019-03-24 00:00:00+00:00  4.5
    654  widget-ts-cookiecutter      654 2019-04-07 00:00:00+00:00  4.0
    655  widget-ts-cookiecutter      655 2019-04-21 00:00:00+00:00  0.0
    656  widget-ts-cookiecutter      656 2019-05-05 00:00:00+00:00  2.5


### Activity over time in each organization

In [16]:
charts = []

for kind, idata in [('issues', issues), ('prs', prs), ('comments', comments)]:
    org_comments = idata.groupby('org').resample('W', on='createdAt').count()['id'].reset_index()
    org_comments = org_comments.groupby('org').rolling(4, on='createdAt').mean().reset_index()

    ch = highlighted_line_plot(org_comments, 'createdAt', 'id', 'org', f"{kind} over time", legend=None, domain=domain)
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [LayerChart({
    data:                   org  level_1                 createdAt     id
    0     binder-examples        0 2017-06-25 00:00:00+00:00    NaN
    1     binder-examples        1 2017-07-02 00:00:00+00:00    NaN
    2     binder-examples        2 2017-07-09 00:00:00+00:00    NaN
    3     binder-examples        3 2017-07-16 00:00:00+00:00   0.25
    4     binder-examples        4 2017-07-23 00:00:00+00:00   0.00
    ...               ...      ...                       ...    ...
    1646       jupyterlab     1646 2019-07-28 00:00:00+00:00  48.75
    1647       jupyterlab     1647 2019-08-04 00:00:00+00:00  43.75
    1648       jupyterlab     1648 2019-08-11 00:00:00+00:00  43.00
    1649       jupyterlab     1649 2019-08-18 00:00:00+00:00  43.75
    1650       jupyterlab     1650 2019-08-25 00:00:00+00:00  33.00
    
    [1651 rows x 4 columns],
    layer: [Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'org',
          legend: None,
          type: 'nominal'
        }),
        opacity: OpacityValue({
          condition: OpacityValue({
            selection: SelectionNot({
              not: 'selector025'
            }),
            value: 0.3
          }),
          value: 1
        }),
        size: SizeValue({
          condition: SizeValue({
            selection: SelectionNot({
              not: 'selector025'
            }),
            value: 2
          }),
          value: 5
        }),
        x: X({
          field: 'createdAt',
          scale: Scale({
            domain: ['2017-06-25', '2019-07-21']
          }),
          type: 'temporal'
        }),
        y: Y({
          field: 'id',
          type: 'quantitative'
        })
      }),
      mark: MarkDef({
        clip: True,
        type: 'line'
      }),
      title: 'issues over time',
      width: 1000
    }), Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'org',
          legend: None,
          type: 'nominal'
        }),
        opacity: ValueDefWithOptionalConditionMarkPropFieldDefnumber({
          value: 0
        }),
        size: ValueDefWithOptionalConditionMarkPropFieldDefnumber({
          value: 300
        }),
        tooltip: Tooltip({
          field: 'org',
          type: 'nominal'
        }),
        x: X({
          field: 'createdAt',
          scale: Scale({
            domain: ['2017-06-25', '2019-07-21']
          }),
          type: 'temporal'
        }),
        y: Y({
          field: 'id',
          type: 'quantitative'
        })
      }),
      mark: MarkDef({
        clip: True,
        type: 'point'
      }),
      selection: {'selector025': SelectionDef({
        fields: ['org'],
        nearest: True,
        on: 'mouseover',
        type: 'single'
      }), 'selector026': SelectionDef({
        bind: 'scales',
        encodings: ['x', 'y'],
        type: 'interval'
      })},
      title: 'issues over time',
      width: 1000
    })]
  }), LayerChart({
    data:                   org  level_1                 createdAt     id
    0     binder-examples        0 2017-10-22 00:00:00+00:00    NaN
    1     binder-examples        1 2017-10-29 00:00:00+00:00    NaN
    2     binder-examples        2 2017-11-05 00:00:00+00:00    NaN
    3     binder-examples        3 2017-11-12 00:00:00+00:00   1.00
    4     binder-examples        4 2017-11-19 00:00:00+00:00   0.75
    ...               ...      ...                       ...    ...
    1208       jupyterlab     1208 2019-08-04 00:00:00+00:00  29.50
    1209       jupyterlab     1209 2019-08-11 00:00:00+00:00  25.50
    1210       jupyterlab     1210 2019-08-18 00:00:00+00:00  26.00
    1211       jupyterlab     1211 2019-08-25 00:00:00+00:00  21.75
    1212          nteract     1212 2019-08-04 00:00:00+00:00    NaN
    
    [1213 rows x 4 columns],
    layer: [Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'org',
          legend: None,

## Time to first response of issues

In [17]:
first_comments = (
    comments.groupby(['org', 'repo', 'issue_id'], as_index=False)                 
    .agg({'id': 'first', 'createdAt': 'first'})
    .rename(columns={'createdAt': 'firstResponse', 'issue_id': 'number'})
    .drop(columns='id')
)

In [18]:
issues_w_response = pd.merge(first_comments, issues, on=['org', 'repo', 'number'])
issues_w_response[['createdAt', 'firstResponse']] = issues_w_response[['createdAt', 'firstResponse']].apply(pd.to_datetime)

In [19]:
issues_w_response['time_to_first_response'] = issues_w_response['firstResponse'] - issues_w_response['createdAt']
issues_w_response['hours_to_first_response'] = issues_w_response['time_to_first_response'].map(lambda a: 24 * a.days + a.seconds / 60 / 60)

In [20]:
charts = []
for iorg in github_orgs:
    print(iorg)
    i_issues = issues_w_response.query('org == @iorg')
    repo_comments = (
        i_issues.groupby('repo')
        .resample(time_bin, on='updatedAt').median()['hours_to_first_response']
        .reset_index()
        .replace(np.nan, 0)
        .groupby('repo').rolling(5, on='updatedAt').mean().reset_index('repo')
    )

    n_plot = 20
    top_repos = repo_comments.groupby('repo').count()['hours_to_first_response'].sort_values(ascending=False).head(n_plot).index.values
    repo_comments = repo_comments.query('repo in @top_repos')
    domain = [fmt_date.format(repo_comments['updatedAt'].min()), fmt_date.format(repo_comments['updatedAt'].max())]
    ch = highlighted_line_plot(repo_comments, 'updatedAt', 'hours_to_first_response', 'repo', f"Hours to first response for {iorg}",
                               legend=None, domain=domain)
    charts.append(ch)
alt.hconcat(*charts)

jupyterhub


jupyter-widgets
jupyter


jupyterlab


ipython


binder-examples


HConcatChart({
  hconcat: [LayerChart({
    data:                         repo                 updatedAt  \
    4               batchspawner 2016-07-10 00:00:00+00:00   
    5               batchspawner 2016-07-24 00:00:00+00:00   
    6               batchspawner 2016-08-07 00:00:00+00:00   
    7               batchspawner 2016-08-21 00:00:00+00:00   
    8               batchspawner 2016-09-04 00:00:00+00:00   
    ...                      ...                       ...   
    2048  zero-to-jupyterhub-k8s 2019-07-07 00:00:00+00:00   
    2049  zero-to-jupyterhub-k8s 2019-07-21 00:00:00+00:00   
    2050  zero-to-jupyterhub-k8s 2019-08-04 00:00:00+00:00   
    2051  zero-to-jupyterhub-k8s 2019-08-18 00:00:00+00:00   
    2052  zero-to-jupyterhub-k8s 2019-09-01 00:00:00+00:00   
    
          hours_to_first_response  
    4                         NaN  
    5                         NaN  
    6                         NaN  
    7                         NaN  
    8                   22.398333  
    ...                       ...  
    2048               439.230917  
    2049               486.551833  
    2050               544.098639  
    2051               529.202972  
    2052              1147.504083  
    
    [1576 rows x 3 columns],
    layer: [Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'repo',
          legend: None,
          type: 'nominal'
        }),
        opacity: OpacityValue({
          condition: OpacityValue({
            selection: SelectionNot({
              not: 'selector031'
            }),
            value: 0.3
          }),
          value: 1
        }),
        size: SizeValue({
          condition: SizeValue({
            selection: SelectionNot({
              not: 'selector031'
            }),
            value: 2
          }),
          value: 5
        }),
        x: X({
          field: 'updatedAt',
          scale: Scale({
            domain: ['2015-01-11', '2019-09-01']
          }),
          type: 'temporal'
        }),
        y: Y({
          field: 'hours_to_first_response',
          type: 'quantitative'
        })
      }),
      mark: MarkDef({
        clip: True,
        type: 'line'
      }),
      title: 'Hours to first response for jupyterhub',
      width: 1000
    }), Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'repo',
          legend: None,
          type: 'nominal'
        }),
        opacity: ValueDefWithOptionalConditionMarkPropFieldDefnumber({
          value: 0
        }),
        size: ValueDefWithOptionalConditionMarkPropFieldDefnumber({
          value: 300
        }),
        tooltip: Tooltip({
          field: 'repo',
          type: 'nominal'
        }),
        x: X({
          field: 'updatedAt',
          scale: Scale({
            domain: ['2015-01-11', '2019-09-01']
          }),
          type: 'temporal'
        }),
        y: Y({
          field: 'hours_to_first_response',
          type: 'quantitative'
        })
      }),
      mark: MarkDef({
        clip: True,
        type: 'point'
      }),
      selection: {'selector031': SelectionDef({
        fields: ['repo'],
        nearest: True,
        on: 'mouseover',
        type: 'single'
      }), 'selector032': SelectionDef({
        bind: 'scales',
        encodings: ['x', 'y'],
        type: 'interval'
      })},
      title: 'Hours to first response for jupyterhub',
      width: 1000
    })]
  }), LayerChart({
    data:                        repo                 updatedAt  hours_to_first_response
    0                ipyleaflet 2018-04-08 00:00:00+00:00                      NaN
    1                ipyleaflet 2018-04-22 00:00:00+00:00                      NaN
    2                ipyleaflet 2018-05-06 00:00:00+00:00                      NaN
    3                ipyleaflet 2018-05-20 00:00:00+00:00                      NaN
    4                ipyleaflet 2018-06-03 00:00:00+00:00              1657.512167
    ..               

In [21]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>